# Re-run active editors skin statistics [T180860](https://phabricator.wikimedia.org/T180860)

In [2]:
import pandas as pd
import numpy as np

import datetime as dt

from wmfdata import hive, mariadb

To try this out on a smaller dataset, we will first begin by taking all users that were active editors in the first quarter of 2019 i.e. 2019Q3.  
We will use their user_id and query the mariadb table user_properties to get their skin preferences. 

In [3]:
#Active Editors for 2019 Q3

active_ed_2019Q3=hive.run("""
select
        cast(month as date) as month,
        wiki,
        user_name,
        user_id,
        sum(content_edits) as content_edits
    from neilpquinn.editor_month
    where
        month >= "2019-01-01" and month < "2019-04-01" --2019Q3
        and user_id != 0
        and content_edits >= 5 and
    not bot_by_group and
    user_name not regexp "bot\\b"
    group by month, wiki, user_name, user_id
""")

In [5]:
active_ed_2019Q3.head()

,month,wiki,user_name,user_id,content_edits
0,2019-01-01,acewiki,Iqbalhafidh,5850,5
1,2019-01-01,afwiki,Burgert Behr,2401,201
2,2019-01-01,afwiki,Ercé,47759,7
3,2019-01-01,afwiki,Kovu777,107607,9
4,2019-01-01,alswiki,Eruedin,108,11


In [6]:
#Querying user_properties for getting the skin preferences set by the active editors we got in the above query

query='''
Select 
    up_value as skin, 
    count(*) as users
    from user_properties
    where up_user in ({users})
    and up_property = "skin"
    group by up_value
'''

In [7]:
# Looping through each wiki for the list of users

up_skin_2019Q3=list()
for wiki in active_ed_2019Q3['wiki'].unique():
    up_skin_2019Q3.append(
    mariadb.run(query.format(
        users = ','.join([str(u) for u in active_ed_2019Q3.loc[
            active_ed_2019Q3['wiki'] == wiki]['user_id']])
    ), wiki))


In [8]:
skin_2019Q3 = pd.concat(up_skin_2019Q3)

In [10]:
skin_2019Q3.head()

,skin,users
0,monobook,1
1,vector,1
0,monobook,1
1,vector,3
0,modern,1


In [11]:
#When skin is null consider that as 'Vector'
skin_2019Q3['skin']=np.where((skin_2019Q3.skin==''),'vector',skin_2019Q3.skin) 

In [12]:
#When skin is 0 consider that as 'Vector'
skin_2019Q3['skin']=np.where((skin_2019Q3.skin==0),'vector',skin_2019Q3.skin) 

In [13]:
#When skin is 2 consider that as 'cologne blue'
skin_2019Q3['skin']=np.where((skin_2019Q3.skin==2),'cologneblue',skin_2019Q3.skin) 

#### Number of users for each skin type


In [14]:
user_skin_2019Q3=skin_2019Q3.groupby('skin').sum()
user_skin_2019Q3

,users
skin,
chick,7
cologneblue,186
minerva,121
modern,735
monobook,6680
myskin,2
nostalgia,7
simple,5
timeless,671


In [15]:
pct_user_skin_2019Q3=(100. * user_skin_2019Q3 / user_skin_2019Q3.sum()).round(1).astype(str) + '%'

#### Percentage of users for each skin type

In [16]:
pct_user_skin_2019Q3

,users
skin,
chick,0.1%
cologneblue,1.4%
minerva,0.9%
modern,5.5%
monobook,49.6%
myskin,0.0%
nostalgia,0.1%
simple,0.0%
timeless,5.0%


Looks like around 50% of active editors in Q3 preffered using Monobook skin.   
Now let's get the skin preference for the entire year of 2019 

In [17]:
#Active Editors for 2019 

active_ed_2019=hive.run("""
select
        cast(month as date) as month,
        wiki,
        user_name,
        user_id,
        sum(content_edits) as content_edits
    from neilpquinn.editor_month
    where
        month >= "2019-01-01" and month < "2020-01-01"
        and user_id != 0
        and content_edits >= 5 and
    not bot_by_group and
    user_name not regexp "bot\\b"
    group by month, wiki, user_name, user_id
""")

In [19]:
# Looping through each wiki for the list of users

up_skin_2019=list()
for wiki in active_ed_2019['wiki'].unique():
    up_skin_2019.append(
    mariadb.run(query.format(
        users = ','.join([str(u) for u in active_ed_2019.loc[
            active_ed_2019['wiki'] == wiki]['user_id']])
    ), wiki))


In [20]:
skin_2019 = pd.concat(up_skin_2019)

In [22]:
#When skin is null consider it as 'Vector'

skin_2019['skin']=np.where((skin_2019.skin==''),'vector',skin_2019.skin) 

In [23]:
#When skin is 0 consider it as 'Vector'

skin_2019['skin']=np.where((skin_2019.skin==0),'vector',skin_2019.skin) 

In [27]:
#When skin is simple consider it as 'Vector'

skin_2019['skin']=np.where((skin_2019.skin=='simple'),'vector',skin_2019.skin) 

In [24]:
#When skin is 2 consider it as 'cologneblue'

skin_2019['skin']=np.where((skin_2019.skin==2),'cologneblue',skin_2019.skin) 

#### Number of users for each skin type in 2019

In [28]:
user_skin_2019=skin_2019.groupby('skin').sum()
user_skin_2019

,users
skin,
chick,17
cologneblue,391
minerva,361
minervaneue,2
modern,1314
monobook,11895
myskin,8
nostalgia,17
standard,2


#### Percentage of users for each skin type in 2019

In [41]:
pct_user_skin_2019=(100. * user_skin_2019 / user_skin_2019.sum()).round(1).astype(str) + '%'
pct_user_skin_2019

,users
skin,
chick,0.1%
cologneblue,1.4%
minerva,1.3%
minervaneue,0.0%
modern,4.7%
monobook,42.8%
myskin,0.0%
nostalgia,0.1%
standard,0.0%


We see that Vector and Monobook were the top preferences of most active editors in 2019.  

Now we will look at skin preferences of "Very" active editors i.e. users who have content edits >=600

In [30]:
# Very Active Editors for 2019 

very_active_ed_2019=hive.run("""
select
        cast(month as date) as month,
        wiki,
        user_name,
        user_id,
        sum(content_edits) as content_edits
    from neilpquinn.editor_month
    where
        month >= "2019-01-01" and month < "2020-01-01" 
        and user_id != 0
        and content_edits >= 600 and
    not bot_by_group and
    user_name not regexp "bot\\b"
    group by month, wiki, user_name, user_id
""")

In [31]:
# Looping through each wiki for the list of users

active_up_skin_2019=list()
for wiki in very_active_ed_2019['wiki'].unique():
    active_up_skin_2019.append(
    mariadb.run(query.format(
        users = ','.join([str(u) for u in very_active_ed_2019.loc[
            very_active_ed_2019['wiki'] == wiki]['user_id']])
    ), wiki))


In [32]:
very_active_skin_2019 = pd.concat(active_up_skin_2019)

In [33]:
#When skin is null consider it as 'Vector'

very_active_skin_2019['skin']=np.where((very_active_skin_2019.skin==''),'vector',very_active_skin_2019.skin) 

In [34]:
#When skin is 0 consider it as 'Vector'

very_active_skin_2019['skin']=np.where((very_active_skin_2019.skin==0),'vector',very_active_skin_2019.skin) 

In [35]:
#When skin is 'simple' consider it as 'Vector'

very_active_skin_2019['skin']=np.where((very_active_skin_2019.skin=='simple'),'vector',very_active_skin_2019.skin)

In [36]:
#When skin is 2 consider it as 'cologneblue'

very_active_skin_2019['skin']=
    np.where((very_active_skin_2019.skin==2),'cologneblue',very_active_skin_2019.skin) 

#### Number of very active editors for each skin type in 2019

In [37]:
active_user_skin_2019=very_active_skin_2019.groupby('skin').sum()
active_user_skin_2019

,users
skin,
cologneblue,10
minerva,5
modern,73
monobook,855
timeless,69
vector,160


#### Percentage of very active users for each skin type in 2019

In [39]:
pct_active_skin_2019=(100. * active_user_skin_2019 / active_user_skin_2019.sum()).round(1).astype(str) + '%'
pct_active_skin_2019

,users
skin,
cologneblue,0.9%
minerva,0.4%
modern,6.2%
monobook,73.0%
timeless,5.9%
vector,13.7%


Looking at the drop in Vector skin preference for very active editors, there could be a high possibility that the database was cleared out of Vector to save unnecessary storage. This needs to be confirmed though. 